In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.01
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/2000shot/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
#     teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.001000
Train | 20/20 | Loss:1.0901 | MainLoss:1.0901 | Alpha:0.0237 | SPLoss:44.5269 | CLSLoss:4.7168 | top1:50.3250 | AUROC:0.0000
Test | 161/20 | Loss:0.6942 | MainLoss:0.6942 | SPLoss:95.9125 | CLSLoss:4.4070 | top1:50.8474 | AUROC:0.5211
Test | 39/20 | Loss:0.6148 | MainLoss:0.6148 | SPLoss:95.9126 | CLSLoss:4.4069 | top1:85.7564 | AUROC:0.9546

Epoch: [2 | 1000] LR: 0.001300
Train | 20/20 | Loss:0.7014 | MainLoss:0.7014 | Alpha:0.0245 | SPLoss:127.5034 | CLSLoss:4.3124 | top1:50.0250 | AUROC:0.0000
Test | 161/20 | Loss:0.7070 | MainLoss:0.7070 | SPLoss:148.9389 | CLSLoss:4.2572 | top1:50.0031 | AUROC:0.5793
Test | 39/20 | Loss:0.6439 | MainLoss:0.6439 | SPLoss:148.9388 | CLSLoss:4.2572 | top1:50.0000 | AUROC:0.9833

Epoch: [3 | 1000] LR: 0.001600
Train | 20/20 | Loss:0.6939 | MainLoss:0.6939 | Alpha:0.0247 | SPLoss:157.4898 | CLSLoss:4.2092 | top1:50.8500 | AUROC:0.0000
Test | 161/20 | Loss:0.6893 | MainLoss:0.6893 | SPLoss:164.1765 | CLSLoss:4.1750 | top1

Test | 39/20 | Loss:0.7814 | MainLoss:0.7814 | SPLoss:907.2732 | CLSLoss:4.9789 | top1:54.5256 | AUROC:0.6959

Epoch: [23 | 1000] LR: 0.003999
Train | 20/20 | Loss:0.3530 | MainLoss:0.3530 | Alpha:0.0238 | SPLoss:921.1432 | CLSLoss:5.0677 | top1:83.9500 | AUROC:0.0000
Test | 161/20 | Loss:0.1081 | MainLoss:0.1081 | SPLoss:937.5579 | CLSLoss:5.1204 | top1:97.9377 | AUROC:0.9991
Test | 39/20 | Loss:0.8538 | MainLoss:0.8538 | SPLoss:937.5574 | CLSLoss:5.1204 | top1:55.3205 | AUROC:0.6867

Epoch: [24 | 1000] LR: 0.003999
Train | 20/20 | Loss:0.3647 | MainLoss:0.3647 | Alpha:0.0235 | SPLoss:958.5205 | CLSLoss:4.9970 | top1:83.6500 | AUROC:0.0000
Test | 161/20 | Loss:0.2742 | MainLoss:0.2742 | SPLoss:985.3179 | CLSLoss:4.7119 | top1:89.5078 | AUROC:0.9990
Test | 39/20 | Loss:0.6573 | MainLoss:0.6573 | SPLoss:985.3168 | CLSLoss:4.7119 | top1:63.2179 | AUROC:0.6878

Epoch: [25 | 1000] LR: 0.003998
Train | 20/20 | Loss:0.3439 | MainLoss:0.3439 | Alpha:0.0242 | SPLoss:1003.5248 | CLSLoss:4.7546 

Test | 161/20 | Loss:0.1549 | MainLoss:0.1549 | SPLoss:1722.3398 | CLSLoss:5.6248 | top1:94.2150 | AUROC:0.9980
Test | 39/20 | Loss:0.9595 | MainLoss:0.9595 | SPLoss:1722.3396 | CLSLoss:5.6248 | top1:54.9231 | AUROC:0.5691

Epoch: [45 | 1000] LR: 0.003989
Train | 20/20 | Loss:0.2582 | MainLoss:0.2582 | Alpha:0.0244 | SPLoss:1739.0282 | CLSLoss:5.5776 | top1:88.7750 | AUROC:0.0000
Test | 161/20 | Loss:0.2006 | MainLoss:0.2006 | SPLoss:1758.1060 | CLSLoss:5.5277 | top1:91.9813 | AUROC:0.9987
Test | 39/20 | Loss:0.9216 | MainLoss:0.9216 | SPLoss:1758.1099 | CLSLoss:5.5277 | top1:55.7436 | AUROC:0.5820

Epoch: [46 | 1000] LR: 0.003989
Train | 20/20 | Loss:0.2504 | MainLoss:0.2504 | Alpha:0.0240 | SPLoss:1773.4556 | CLSLoss:5.5903 | top1:89.7000 | AUROC:0.0000
Test | 161/20 | Loss:0.2107 | MainLoss:0.2107 | SPLoss:1790.6199 | CLSLoss:5.5877 | top1:91.4922 | AUROC:0.9981
Test | 39/20 | Loss:0.9186 | MainLoss:0.9186 | SPLoss:1790.6198 | CLSLoss:5.5877 | top1:55.5769 | AUROC:0.5831

Epoch: [47

Train | 20/20 | Loss:0.1684 | MainLoss:0.1684 | Alpha:0.0240 | SPLoss:1995.6190 | CLSLoss:6.2468 | top1:93.3750 | AUROC:0.0000
Test | 161/20 | Loss:0.1564 | MainLoss:0.1564 | SPLoss:1995.3651 | CLSLoss:6.2811 | top1:94.0748 | AUROC:0.9989
Test | 39/20 | Loss:1.1568 | MainLoss:1.1568 | SPLoss:1995.3687 | CLSLoss:6.2811 | top1:54.7564 | AUROC:0.5670

Epoch: [67 | 1000] LR: 0.000397
Train | 20/20 | Loss:0.1690 | MainLoss:0.1690 | Alpha:0.0236 | SPLoss:1995.0297 | CLSLoss:6.2969 | top1:93.0500 | AUROC:0.0000
Test | 161/20 | Loss:0.1819 | MainLoss:0.1819 | SPLoss:1995.1442 | CLSLoss:6.2995 | top1:92.9688 | AUROC:0.9989
Test | 39/20 | Loss:1.1454 | MainLoss:1.1454 | SPLoss:1995.1453 | CLSLoss:6.2995 | top1:54.7308 | AUROC:0.5694

Epoch: [68 | 1000] LR: 0.000397
Train | 20/20 | Loss:0.1670 | MainLoss:0.1670 | Alpha:0.0235 | SPLoss:1995.1660 | CLSLoss:6.3104 | top1:93.1000 | AUROC:0.0000
Test | 161/20 | Loss:0.1474 | MainLoss:0.1474 | SPLoss:1994.7043 | CLSLoss:6.3170 | top1:94.3645 | AUROC:0.

Test | 39/20 | Loss:1.2188 | MainLoss:1.2188 | SPLoss:2010.0250 | CLSLoss:6.6048 | top1:54.6538 | AUROC:0.5664

Epoch: [88 | 1000] LR: 0.000394
Train | 20/20 | Loss:0.1689 | MainLoss:0.1689 | Alpha:0.0242 | SPLoss:2010.8750 | CLSLoss:6.6110 | top1:93.0250 | AUROC:0.0000
Test | 161/20 | Loss:0.1432 | MainLoss:0.1432 | SPLoss:2011.8561 | CLSLoss:6.5996 | top1:94.6012 | AUROC:0.9990
Test | 39/20 | Loss:1.2149 | MainLoss:1.2149 | SPLoss:2011.8549 | CLSLoss:6.5996 | top1:54.3333 | AUROC:0.5617

Epoch: [89 | 1000] LR: 0.000394
Train | 20/20 | Loss:0.1545 | MainLoss:0.1545 | Alpha:0.0235 | SPLoss:2012.5145 | CLSLoss:6.5900 | top1:94.0500 | AUROC:0.0000
Test | 161/20 | Loss:0.1621 | MainLoss:0.1621 | SPLoss:2013.4758 | CLSLoss:6.5920 | top1:93.8349 | AUROC:0.9990
Test | 39/20 | Loss:1.1827 | MainLoss:1.1827 | SPLoss:2013.4736 | CLSLoss:6.5920 | top1:54.6154 | AUROC:0.5643

Epoch: [90 | 1000] LR: 0.000394
Train | 20/20 | Loss:0.1519 | MainLoss:0.1519 | Alpha:0.0243 | SPLoss:2013.4255 | CLSLoss:

Train | 20/20 | Loss:0.1440 | MainLoss:0.1440 | Alpha:0.0241 | SPLoss:2034.2228 | CLSLoss:6.9468 | top1:94.3250 | AUROC:0.0000
Test | 161/20 | Loss:0.1737 | MainLoss:0.1737 | SPLoss:2035.1388 | CLSLoss:6.9505 | top1:93.6231 | AUROC:0.9989
Test | 39/20 | Loss:1.2747 | MainLoss:1.2747 | SPLoss:2035.1350 | CLSLoss:6.9505 | top1:54.3462 | AUROC:0.5611

Epoch: [110 | 1000] LR: 0.000391
Train | 20/20 | Loss:0.1348 | MainLoss:0.1348 | Alpha:0.0249 | SPLoss:2036.5786 | CLSLoss:6.9706 | top1:94.6750 | AUROC:0.0000
Test | 161/20 | Loss:0.1746 | MainLoss:0.1746 | SPLoss:2038.4398 | CLSLoss:6.9913 | top1:93.6542 | AUROC:0.9990
Test | 39/20 | Loss:1.3044 | MainLoss:1.3044 | SPLoss:2038.4388 | CLSLoss:6.9913 | top1:54.2821 | AUROC:0.5580

Epoch: [111 | 1000] LR: 0.000390
Train | 20/20 | Loss:0.1440 | MainLoss:0.1440 | Alpha:0.0240 | SPLoss:2038.2325 | CLSLoss:7.0163 | top1:94.5750 | AUROC:0.0000
Test | 161/20 | Loss:0.1617 | MainLoss:0.1617 | SPLoss:2038.2640 | CLSLoss:7.0113 | top1:94.1308 | AUROC: